In [51]:
import pandas as pd
import numpy as np

#### Read weather data

In [52]:
df_weather = pd.read_csv('../Dataset/weather/weather.csv')

In [53]:
df_weather.head(5)

,date,time,dewptm,hum,pressurem,tempm,wdird,wspdm
0,2014-08-01,0,11.666667,64.000000,1012.0,18.000000,213.333333,6.800000
1,2014-08-01,1,13.333333,70.000000,1012.0,18.333333,210.000000,7.400000
2,2014-08-01,2,14.333333,75.333333,1012.0,18.666667,213.333333,9.866667
3,2014-08-01,3,15.000000,79.666667,1012.0,18.000000,226.666667,9.300000
4,2014-08-01,4,14.666667,80.333333,1012.0,17.666667,223.333333,8.066667


### Merge weather data to dataset

In [54]:
#handle each file
def handle_detail_file(df_detail):
    df_detail['date'] = df_detail['TIMESTAMP'].apply(lambda x: x.split('T')[0])
    df_detail['time'] = df_detail['TIMESTAMP'].apply(lambda x: x.split('T')[1].split(':')[0])
    df_detail.drop(columns=['status','extID','medianMeasuredTime','TIMESTAMP'], inplace= True)
    df_group = df_detail.groupby(['date', 'time']).agg({'avgMeasuredTime': 'mean','vehicleCount': 'sum'}).reset_index()
    return df_group

In [55]:
#merge with weather data
def merge_with_weather(df_detail, df_weather):
    df_detail['time'] = pd.to_numeric(df_detail['time'])
    return pd.merge(df_detail, df_weather, on=['date', 'time'], how='left')


In [56]:
#merge with cultural event data
def merge_cultural_event(df_merge_weather):
    df_cultural_event = pd.read_csv('../Dataset/cultural_events_aarhus.csv', header=None, usecols=[12,16])
    df_cultural_event.columns = ['timestamp', 'event']
    df_cultural_event['date'] = df_cultural_event['timestamp'].apply(lambda x: x.split('T')[0])
    df_cultural_event['time'] = df_cultural_event['timestamp'].apply(lambda x: x.split('T')[1].split(':')[0])
    df_cultural_event.drop(columns=['timestamp'], inplace= True)
    df_cultural_event['time'] = pd.to_numeric(df_cultural_event['time'])
    df_merge = pd.merge(df_merge_weather, df_cultural_event, on=['date', 'time'], how='left')
    df_merge['event'] = df_merge['event'].replace(np.nan, 'No Event')
    return df_merge

In [57]:
#read all file in traffic_feb_june folder
import os
files = os.listdir('../Dataset/traffic_june_sep/')
# dff = pd.DataFrame(columns=["status","avgMeasuredTime","avgSpeed","extID","medianMeasuredTime","TIMESTAMP","vehicleCount","_id","REPORT_ID"])
for filename in files:
    dff_temp = pd.read_csv('../Dataset/traffic_june_sep/'+filename)
    dff_detail = handle_detail_file(dff_temp)
    dff_merge_weather = merge_with_weather(dff_detail, df_weather)
    df_merge = merge_cultural_event(dff_merge_weather)
    df_merge.to_csv('../Dataset/weather_merged1/add_feature_'+filename,index = False)